<a href="https://colab.research.google.com/github/chris-kehl/Coursera_Computer_Vision/blob/main/TensorFlow_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple object detection in TensorFlow

*   explore the TensorFlow hub for object detection models. 
*   load models in your workspace
*   preprocess and image in inference
*   run inference on models and inspect the output





In [1]:
# Import models
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

**Download the model from Tensorflow Hub**

*   Look at the image object detection subcategory
*   Select a inception resnet version 2 model
*   Select a mobilenet version 2 model

In [2]:
# inception resnet version 2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

**Load the model**

In [3]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


**Choose the default signature**

Show what signature to use when running the model.

In [4]:
# take a look at the available signatures for this particular model
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction pruned(images) at 0x7F2F9E336E90>}))

Chose the default signature for object detection.  

*   For object detection its default signature will accept a batch of image tensors and output a dictionary describing the objects detected, which is what you'll want here.

In [5]:
detector = model.signatures['default']

**download and resize image**

This function downloads an image specified by a given "url", preprocess it, and then saves it to disk.

In [8]:
'''
Fetches an image online, resizes it and saves it locally.

Args:
    url (string) -- link to the image
    new_width (int) -- size in pixels used for resizing the width of an image.
    new_height (int) -- size in pixels used for resizing the length of an image.

Returns:
    (string) -- path to the saved image
'''
def download_and_resize_image(url, new_width=256, new_height=256):
    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    # opens the given URL
    response = urlopen(url)
    # reads the image data in memory buffer
    image_data = response.read()
    # puts the image data in memory buffer
    image_data = BytesIO(image_data)
    # opens the image
    pil_image = Image.open(image_data)
    # resizes the image. Crop if aspect ratio is different
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")
    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)

    print("Image downloaded to %s." % filename)

    return filename

**Download and preprocess an image**

use download_and_resize_image to get a sample image online and save it locally. 

In [9]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"
# download using the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmp7d30lwid.jpg.


**run_detector**

This function will take in the object detection model detector and the path to a sample image, then use the model to detect objects and display its predicted class categories and detection boxes

run_detector uses load_image to convert the image into a tensor

In [11]:
'''
Loads a JPEG image and converts it to a tensor.

Args:
    Path (string) -- path to a locally saved JPEG
Returns:
    (tensor) -- an image tensor
'''

def load_img(path):
    # read the file
    img = tf.io.read_file(path)
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    return img

'''
run_detector runs inference on a local file using an object detection model.

Args:
    detector (model) -- an object detection model loaded from the TF Hub
    path (string) -- path to an image saved locally
'''
def run_detector(detector, path):
# load an image tensor from a local file path
    img = load_img(path)

    # add a batch dimension in front of the tensor
    converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

    # run inference using the modle
    result = detector(converted_img)

    # Save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])

**Run Inference on the image**



*   Print the number of objects found followed by three lists
    *   The detection scores of each object found
    *   The class of each object found
    *   The bounding boxes of each object






In [13]:
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.65321714 0.6105055  0.60152465 0.59255624 0.5917768  0.58154947
 0.5505308  0.49575484 0.47424984 0.47322112 0.44066438 0.40511352
 0.3980382  0.39406413 0.3714846  0.3615593  0.3615083  0.34689054
 0.33362454 0.31252864 0.28878096 0.25758332 0.25748888 0.25196084
 0.24782084 0.23412465 0.20432039 0.20324546 0.1798825  0.1796485
 0.17374535 0.16431475 0.16031165 0.15895209 0.15620255 0.15468812
 0.14754422 0.13622648 0.12740108 0.12555623 0.12102751 0.11813115
 0.11387312 0.11229102 0.11129206 0.09718706 0.09137283 0.08975969
 0.08880271 0.08633585 0.08337469 0.0809515  0.07988653 0.07741486
 0.07732126 0.07631026 0.07507819 0.07386063 0.07233284 0.07204071
 0.0711016  0.06935622 0.06825593 0.06427879 0.06248593 0.06226419
 0.06211162 0.05940076 0.05798756 0.05784443 0.05725611 0.05346691
 0.05304342 0.0524287  0.04892577 0.04813066 0.0457845  0.04424164
 0.04337563 0.0427781  0.04262616 0.04163295 0.04081752 0.03975986
 0.03946014 0.03944279 0.03864    0.0376908 